### Imports and path

In [ ]:
# Imports
import numpy as np
import pandas as pd
import os

### Calculate engineering constants from stiffness coefficients

In [ ]:
# Load the CSV file
mainpath = os.getcwd() + '/'
structure = 'EW3'
file_path = mainpath + structure + '/' + structure + '_stiffness_coeff.csv'
df = pd.read_csv(file_path)  # Use ',' if CSV is comma-separated
metadata = df[["structure", "mc", "lig"]].copy()
display(df)

# Extract stiffness coefficients
C = np.array([
    [df["C11"][0], df["C12"][0], df["C13"][0], 0, 0, 0],
    [df["C12"][0], df["C22"][0], df["C23"][0], 0, 0, 0],
    [df["C13"][0], df["C23"][0], df["C33"][0], 0, 0, 0],
    [0, 0, 0, df["C44"][0], 0, 0],
    [0, 0, 0, 0, df["C55"][0], 0],
    [0, 0, 0, 0, 0, df["C66"][0]],
])

# Compute compliance matrix (inverse of stiffness matrix)
S = np.linalg.inv(C)

# Extract Engineering Constants
E1 = 1 / S[0, 0]  # Young's modulus in direction 1
E2 = 1 / S[1, 1]  # Young's modulus in direction 2
E3 = 1 / S[2, 2]  # Young's modulus in direction 3
ν12 = -S[0, 1] * E1  # Poisson's ratio ν12
ν13 = -S[0, 2] * E1  # Poisson's ratio ν13
ν23 = -S[1, 2] * E2  # Poisson's ratio ν23
G12 = 1 / S[5, 5]  # Shear modulus G12
G23 = 1 / S[3, 3]  # Shear modulus G13
G13 = 1 / S[4, 4]  # Shear modulus G23

# Display results
#print("Compliance Matrix (S):\n", S)
print("\nEngineering Constants:")
print(f"E1 = {E1:.4f}, E2 = {E2:.4f}, E3 = {E3:.4f}")
print(f"ν12 = {ν12:.4f}, ν13 = {ν13:.4f}, ν23 = {ν23:.4f}")
print(f"G12 = {G12:.4f}, G13 = {G13:.4f}, G23 = {G23:.4f}")

constants_df = pd.DataFrame({
    "E1": [E1], "E2": [E2], "E3": [E3],
    "nu23": [ν23], "nu13": [ν13], "nu12": [ν12],
    "G23": [G23], "G13": [G13], "G12": [G12]
})
final_df = pd.concat([metadata, constants_df], axis=1)
savepath = mainpath + structure + '/' + structure + '_eng_const.csv'
final_df.to_csv(savepath, index=False)